In [1]:
import pandas as pd
import json
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from textblob import TextBlob
import string

In [2]:
data_file = open("data/yelp_academic_dataset_review.json", encoding = "utf8")
reviews_data = []
c = 0
for line in data_file:
    reviews_data.append(json.loads(line))
    c+=1
    if c == 1000000:
        break

reviews_df = pd.DataFrame(reviews_data)

data_file.close()

In [3]:
data_file = open("data/yelp_academic_dataset_business.json", encoding = "utf8")
business_data = []
for line in data_file:
    business_data.append(json.loads(line))

business_df = pd.DataFrame(business_data)

data_file.close()

In [4]:
# reviews_df.sort_values('date')

In [5]:
# Filter out reviews for restaurants not in PA 
business_list = business_df[business_df['state']=='PA']['business_id']
df = reviews_df[reviews_df['business_id'].isin(business_list)]

In [6]:
def split_time_to_feature(df):

    df['date'] =  pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')
    
    df['year'] = pd.DatetimeIndex(df['date']).year
    df['month'] = pd.DatetimeIndex(df['date']).month
    df['day'] = pd.DatetimeIndex(df['date']).day
    df['hour'] = pd.DatetimeIndex(df['date']).hour
    df['min'] = pd.DatetimeIndex(df['date']).minute
    df['sec'] = pd.DatetimeIndex(df['date']).second

split_time_to_feature(df)

/var/folders/ph/709k81096ysfhdprrztggzfr0000gn/T/ipykernel_30731/789486673.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] =  pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S')
/var/folders/ph/709k81096ysfhdprrztggzfr0000gn/T/ipykernel_30731/789486673.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = pd.DatetimeIndex(df['date']).year
/var/folders/ph/709k81096ysfhdprrztggzfr0000gn/T/ipykernel_30731/789486673.py:6: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [7]:
# Extract reivews left in 2000-2018
df = df.loc[(df['year'] >= 2000) & (df['year'] <= 2018)]

### Feature engineering

In [8]:
# Creating a new column in the dataset for the length of the reviews
df['length_of_reviews'] = df['text'].apply(len)

/var/folders/ph/709k81096ysfhdprrztggzfr0000gn/T/ipykernel_30731/1576461174.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['length_of_reviews'] = df['text'].apply(len)


In [9]:
# Creating a new column in the dataset for the number of words in the reviews
df['num_of_words'] = df['text'].apply(lambda str:len(nltk.word_tokenize(str)))

/var/folders/ph/709k81096ysfhdprrztggzfr0000gn/T/ipykernel_30731/1114910945.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['num_of_words'] = df['text'].apply(lambda str:len(nltk.word_tokenize(str)))


In [10]:
# Creating a new column in the dataset for the number of sentences in the reviews
df['num_of_sentences'] = df['text'].apply(lambda paragraph:len(nltk.sent_tokenize(paragraph)))

/var/folders/ph/709k81096ysfhdprrztggzfr0000gn/T/ipykernel_30731/1147211261.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['num_of_sentences'] = df['text'].apply(lambda paragraph:len(nltk.sent_tokenize(paragraph)))


In [11]:
df['capital_words'] = df['text'].apply(lambda sen:len(re.findall(r'\b[A-Z]+\b', sen)))
df['capital_words_ratio'] = df['capital_words']/df['num_of_words']
df.drop(columns='capital_words', inplace = True)

/var/folders/ph/709k81096ysfhdprrztggzfr0000gn/T/ipykernel_30731/1550109840.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['capital_words'] = df['text'].apply(lambda sen:len(re.findall(r'\b[A-Z]+\b', sen)))
/var/folders/ph/709k81096ysfhdprrztggzfr0000gn/T/ipykernel_30731/1550109840.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['capital_words_ratio'] = df['capital_words']/df['num_of_words']
/var/folders/ph/709k81096ysfhdprrztggzfr0000gn/T/ipykernel_30731/1550109840.py:3: SettingWithCopyWarni

### Dealing with texts in reviews

In [12]:
# Cleaning the reviews - stemed, remove stopwords and punctuation
def clean_text(text):

    ## Remove puncuation
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    ## Remove stop words
    nostop = [word for word in nopunc.split() if word.lower() not in stopwords.words('english') and len(word) >= 3]
    text = ' '.join(nostop)
   
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)    
    
    ps = PorterStemmer()
    text = ps.stem(text)

    lemmatizer = WordNetLemmatizer()
    text = lemmatizer.lemmatize(text)

    return text

df['cleaned_text'] = df['text'].apply(clean_text)

In [13]:
# Sentiment analysis using textblob 
df['sentiment_polarity'] = df['cleaned_text'].apply(lambda w:TextBlob(w).polarity)
df['sentiment_subjectivity'] = df['cleaned_text'].apply(lambda w:TextBlob(w).subjectivity)

In [14]:
df.to_csv('data/cleaned_review_data.csv')